In [1]:
pylab inline

/Library/Python/2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


In [2]:
try:
    reload(dc)
    print  'dc reloaded'
except Exception:
    print 'dc not loaded yet'
    

dc not loaded yet


In [3]:
import dnnConnectivityAndMemoryAllocation as dc

# Create memory
memory = dc.Memory(2,32,8,4096)

# Create DNN
network = dc.Network()
#                                    X    Y    Z    Kx   Ky   Kz   stride
network.addLayer('Input',          224, 224,    3                      ) #    3 
network.addLayer('Convolutional',   55,  55,   96,   11,  11,    3,   4 ) #   96,
network.addLayer('Convolutional',   27,  27,  256,    5,   5,   96,   2 ) #  256,
# network.addLayer('Convolutional',   13,  13,  384,    3,   3,  256,   2 ) #  384,
# network.addLayer('Convolutional',   13,  13,  384,    3,   3,  384,   1 ) #  384,
# network.addLayer('Fully Connected', 13,  13,  256,    3,   3,  384,   1 ) #  256,
# network.addLayer('Fully Connected',  1,   1, 4096,   13,  13,  256,   1 ) # 4096,
# network.addLayer('Fully Connected',  1,   1, 4096,    1,   1, 4096,   1 ) # 4096,
# network.addLayer('Fully Connected',  1,   1, 1024,    1,   1, 4096,   1 ) # 1024,

#network.addLayer('Input',          224, 224,    3                      ) #    3 
#network.addLayer('Convolutional',   55,  55,   10,   11,  11,    3,   4 ) #   96,
#network.addLayer('Input',           55,  55,    3,                      ) #   96,
#network.addLayer('Convolutional',   27,  27,    5,    5,   5,    3,   2 ) #  256,
#network.addLayer('Convolutional',   13,  13,   10,    3,   3,    5,   2 ) #  384,
#network.addLayer('Convolutional',   13,  13,    8,    3,   3,   10,   1 ) #  384,
#network.addLayer('Fully Connected', 13,  13,    6,    3,   3,    8,   1 ) #  256,
#network.addLayer('Fully Connected',  1,   1,    6,   13,  13,    6,   1 ) # 4096,
#network.addLayer('Fully Connected',  1,   1,    4,    1,   1,    6,   1 ) # 4096,
#network.addLayer('Fully Connected',  1,   1,    4,    1,   1,    4,   1 ) # 1024,

network.assignPEs()






Layer  0  left Kernel offset is  0 , top Kernel offset is  0
Layer  1  left Kernel offset is  -2 , top Kernel offset is  -2
Layer  2  left Kernel offset is  -1 , top Kernel offset is  -1


In [4]:
for l in range(1, network.numberOfLayers):
  network.Layers[l].generateConnections()


Updating Layer 1 connections for features in row :0
Updating Layer 1 connections for features in row :1
Updating Layer 1 connections for features in row :2
Updating Layer 1 connections for features in row :3
Updating Layer 1 connections for features in row :4
Updating Layer 1 connections for features in row :5
Updating Layer 1 connections for features in row :6


KeyboardInterrupt: 

In [ ]:
network.Layers[0].displayTargetPECounts()


In [ ]:
lid = 0
numOfPEs = network.Layers[lid].getTargetPECounts()


In [ ]:
opt = np.get_printoptions()
np.set_printoptions(threshold=np.inf)
#print numOfPEs
np.set_printoptions(threshold=1000)



In [ ]:
#region = np.array([50,65,45,65])
region = np.array([12,20,9,21])
network.Layers[0].displayTargetPECountsRegion(region)

In [ ]:
layerID = 1
for peY in range(network.peY) :
  for peX in range(network.peX) :
    network.peArray.pe[peY][peX].findROI(layerID)
    network.managerArray.manager[peY][peX].memCpyROI(layerID)


In [ ]:
tempZ = network.managerArray.manager[4][4].roiCells[1][1][0][0].Z
tempY = network.managerArray.manager[4][4].roiCells[1][1][0][0].Y
tempX = network.managerArray.manager[4][4].roiCells[1][1][0][0].X
network.managerArray.manager[4][4].roiCells[1][1][0][0]


In [ ]:
print network.managerArray.manager[4][4].roiCells[1][1][0][0]

In [ ]:
network.managerArray.manager[4][4].roiCells[1][1][0][0].memoryLocation

In [ ]:
network.managerArray.manager[4][4].roiCells[1][1][0][0].originalCell

In [ ]:
network.Layers[0].cells[tempZ][tempY][tempX]

In [ ]:
print network.Layers[0].cells[tempZ][tempY][tempX]

In [ ]:
network.Layers[0].cells[tempZ][tempY][tempX].memoryLocation

In [ ]:
network.Layers[0].cells[tempZ][tempY][tempX].copiedTo

In [ ]:
network.managerArray.manager[0][0].allocateMemory(memory,layerID)
for y in range(network.managerArray.manager[0][0].roiCells[layerID][0].__len__()):
  for x in range(network.managerArray.manager[0][0].roiCells[layerID][0][0].__len__()):
    for z in range(network.managerArray.manager[0][0].roiCells[layerID].__len__()):
        print network.managerArray.manager[0][0].roiCells[layerID][z][y][x]


In [ ]:
roiCells = network.managerArray.manager[0][0].roiCells 

initialChannel   = 0
channelIncrement = 1
initialBank      = 0
bankIncrement    = 2
initialPage      = 0
pageIncrement    = 2
initialWord      = 0
wordIncrement    = 1
padWordRadix2    = 'Y'

# if the number of features is not radix-2, do we pad features to align radix-2
numOfFeatures = roiCells[layerID].__len__()
if padWordRadix2 is 'Y' :
  numberOfAllocatedFeatures = 2**(math.ceil(math.log(numOfFeatures,2)))

channel = initialChannel
bank    = initialBank
page    = initialPage
word    = initialWord

for y in range(roiCells[layerID][0].__len__()):
  for x in range(roiCells[layerID][0][0].__len__()):
    for z in range(roiCells[layerID].__len__()):
      roiCells[layerID][z][y][x].memoryLocation.memory  = memory
      roiCells[layerID][z][y][x].memoryLocation.channel = int(channel)
      roiCells[layerID][z][y][x].memoryLocation.bank    = int(bank)
      roiCells[layerID][z][y][x].memoryLocation.page    = int(page)
      roiCells[layerID][z][y][x].memoryLocation.word    = int(word)
      
      channel = channel + channelIncrement
      if channel == (memory.numOfChannels):
        channel = 0
        word = word + wordIncrement
      if padWordRadix2 is 'Y' :
        if word%numberOfAllocatedFeatures == numOfFeatures:
            word = (int(word/numberOfAllocatedFeatures)+1) * numberOfAllocatedFeatures
      if word == (memory.sizeOfPage/WORDSIZE):
        word = 0
        page = page + pageIncrement
      
      if page == (memory.numOfPagesPerBank):
        page = 0
        bank = bank + bankIncrement
      
      if bank == (memory.numOfBanksPerChannel):
        bank = 0
              
for y in range(roiCells[layerID][0].__len__()):
  for x in range(roiCells[layerID][0][0].__len__()):
    for z in range(roiCells[layerID].__len__()):
        print roiCells[layerID][z][y][x]


In [ ]:
print network.Layers[1].cells[0][25][25]


In [ ]:
print network.peArray.pe[3][3].getROI(1)